In [1]:
from dashboard.logic.io import GSHEETS_URL, read_gsheet

df = read_gsheet(GSHEETS_URL, header=None, nrows=11, usecols=[0,1,2], names=['Asset Class', 'Total Value', 'Comments'])
df = df.dropna(how='all')

In [26]:
df

,Asset Class,Total Value,Comments
0,Cryptocurrencies,"$214,383.87",NaN
1,Gold/Silver (physical),"$236,815.54",NaN
2,Gold/Silver (allocated),"$1,216,701.98",NaN
3,Business Equity,"$432,375.75",NaN
4,Real Estate Equity (IOM/UK),"$1,179,431.85",NaN
5,Real Estate Equity (US),"$482,000.00",NaN
6,Stocks,$0.00,WAITING (Q3 2023?)
7,Cash,"$50,000.00",NaN
9,Total (USD),"$3,811,708.99",<- No Debt
10,Monthly Income,"$50,000.00",NaN


In [2]:
from math import pi

from bokeh.io import output_notebook
from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.resources import CDN
from bokeh.embed import components 

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
df1 = df.copy()
df1 = df1.drop("Comments", axis='columns').query("`Asset Class` != 'Monthly Income' & `Total Value` != '#ERROR!'")
df1['Total Value'] = df1['Total Value'].replace(r"[\$,]", "", regex=True).astype(float)
df1['angle'] = df1["Total Value"] / df1["Total Value"].sum() * 2*pi
df1['color'] = Category20c[df1.shape[0]]
df1 = df1.rename(columns={"Total Value": "total_value"})

source = ColumnDataSource(df1)

p = figure(height=350, title="Asset Allocations", toolbar_location=None,
           tools="hover", tooltips="Value: @total_value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Asset Class', source=source)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

js, div = components(p)
print(div)
#show(p)

<div id="86ea2ccd-5627-4c84-9d76-56b3fd3c99e2" data-root-id="p1123" style="display: contents;"></div>


In [9]:
df2 = df1.copy()
df2['cumsum_start'] = df2['angle'].cumsum(axis='rows').shift(1).fillna(0)
df2['cumsum_end'] = df2['angle'].cumsum(axis='rows')
df2 = df2.rename(columns={"Asset Class": 'asset_class'})
df2 = df2.query("asset_class != 'Total (USD)'")
df2

,asset_class,total_value,angle,color,cumsum_start,cumsum_end
0,Cryptocurrencies,214383.87,0.176694,#3182bd,0.000000,0.176694
1,Gold/Silver (physical),236815.54,0.195182,#6baed6,0.176694,0.371876
2,Gold/Silver (allocated),1216701.98,1.002800,#9ecae1,0.371876,1.374676
3,Business Equity,432375.75,0.356362,#c6dbef,1.374676,1.731039
4,Real Estate Equity (IOM/UK),1179431.85,0.972082,#e6550d,1.731039,2.703121
5,Real Estate Equity (US),482000.00,0.397262,#fd8d3c,2.703121,3.100383
6,Stocks,0.00,0.000000,#fdae6b,3.100383,3.100383
7,Cash,50000.00,0.041210,#fdd0a2,3.100383,3.141593


In [42]:
from bokeh.models import ColumnDataSource, HoverTool, Legend, LabelSet, Label, LegendItem
from bokeh.palettes import Category10
import numpy as np


def bokeh_pie_chart(df, x, y, radius, 
    fig_height=350, 
    pallette=Category10,
    pie_line_color='white',
    legend_place='right',
    **fig_kwargs):
    
    # calculate sector start and end angles
    df['angle'] = df[y] / df[y].sum() * 2 * np.pi
    df['cumsum_start'] = df['angle'].cumsum(axis='rows').shift(1).fillna(0)
    df['cumsum_end'] = df['angle'].cumsum(axis='rows')
    
    # calculate y percentages for hover & labels
    df['percentage_number'] = (df[y] / df[y].sum() * 100).round(1)
    df['percentage_hover'] = df['percentage_number'].map('{:,.1f}%'.format)
    df['percentage_label'] = df['percentage_number'].apply(lambda x: "" if x < 5 else f"{x}%")
    
    # project label text coordinates to polar coordinates
    df['label_x_pos'] = np.cos(df['angle'].cumsum() - df['angle'].div(2)) * 3 * radius/4
    df['label_y_pos'] = np.sin(df['angle'].cumsum() - df['angle'].div(2)) * 3 * radius/4
    
    # remove assets that are 0
    df = df[df[y] > 0]
    
    # reset dataframe index to start with 0
    df = df.reset_index(drop=True)
    
    main_source = ColumnDataSource(df)
    
    # init the figure/canvas for the plot
    p = figure(height=fig_height, toolbar_location=None, x_range=(-1, 1.0), **fig_kwargs)
    
    legend_items = []
    
    for idx, color in enumerate(pallette[df.shape[0]]):
        
        source = ColumnDataSource(df.iloc[idx,:].to_frame().T)
        
        # create the glyphs on canvas
        wedge = p.wedge(x=0, y=0, radius=radius, start_angle="cumsum_start", 
                        end_angle="cumsum_end", source=source, line_color=pie_line_color, line_width=1.5,
                        fill_color=color, hover_fill_color=color, hover_line_color='#212529',
                        alpha=0.7, hover_alpha=1, line_alpha=1, hover_line_alpha=1)
        
        label = LabelSet(x='label_x_pos', y='label_y_pos', text='percentage_label',
                         text_font_size='10pt', text_color="black", source=source,
                         text_align='center', text_alpha=0.65, level='glyph')
        
        p.add_layout(label)
        p.add_tools(HoverTool(renderers=[wedge],
                              tooltips=f"""
                              <div>
                                  <p style="margin:0;font-weight:bold;color:grey;">@{x}</p>
                                  <p style="padding:0;margin:0;font-weight:bold;">@{y}{{$0,0.00}} (@percentage_hover)</p>
                              </div>
                              """,
                              mode='mouse'
            )
        )

        legend_items.append(LegendItem(label=df[x][idx], renderers=[wedge]))
    
    # legend
    legend = Legend(items=legend_items, location='center') 
    p.add_layout(legend, place=legend_place)
    p.legend.click_policy="hide"
    
    p.toolbar.active_drag = None
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    p.outline_line_alpha=0
    p.outline_line_width=0
    p.outline_line_color='#212529'
    p.background_fill_color='#212529'
    p.border_fill_color='#212529'
    
    
    show(p)
    
a = bokeh_pie_chart(df2, x='asset_class', y='total_value', radius=0.8, pie_line_color='#212529')